In [ ]:
!pip install -q "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
import pandas as pd
from datasets import Dataset

dataset_df = pd.read_csv('/content/Review_Analysis.csv')
df = dataset_df.sample(frac=1).reset_index(drop=True)
df.head()

,Review of Mobile and Computer Accessories,Sentiment,Extract Feature Keyword
0,The charging cable frayed within a week.,Negative,"Frayed, Charging cable"
1,The durability of this USB-C cable is impressive.,Positive,"Durability, USB-C cable"
2,Bluetooth pairing on this speaker is a nightmare.,Negative,"Bluetooth pairing, Nightmare"
3,"Amazing sound quality on these headphones, but...",Neutral,"Sound quality, Tight"
4,The monitor stand saves a lot of desk space an...,Positive,"Saves desk space, Adjustable"


In [7]:
df['Sentiment_Feature'] = df['Sentiment']+" | "+df['Extract Feature Keyword']

In [8]:
df.head()

,Review of Mobile and Computer Accessories,Sentiment,Extract Feature Keyword,Sentiment_Feature
0,The charging cable frayed within a week.,Negative,"Frayed, Charging cable","Negative | Frayed, Charging cable"
1,The durability of this USB-C cable is impressive.,Positive,"Durability, USB-C cable","Positive | Durability, USB-C cable"
2,Bluetooth pairing on this speaker is a nightmare.,Negative,"Bluetooth pairing, Nightmare","Negative | Bluetooth pairing, Nightmare"
3,"Amazing sound quality on these headphones, but...",Neutral,"Sound quality, Tight","Neutral | Sound quality, Tight"
4,The monitor stand saves a lot of desk space an...,Positive,"Saves desk space, Adjustable","Positive | Saves desk space, Adjustable"


In [9]:
INTRO_BLURB = "Below is an instruction that describes a task. Write response that appropriately completes the request."
INSTRUCTION_KEY = "### Instruction:"
INPUT_KEY = "Review:"
RESPONSE_KEY = "### Response:"
END_KEY = "### End"



PROMPT_WITH_INPUT_FORMAT = """{intro}

{instruction_key}
{instruction}

{input_key}
{input}

{response_key}
{response}

{end_key}""".format(
  intro=INTRO_BLURB,
  instruction_key=INSTRUCTION_KEY,
  instruction="{instruction}",
  input_key=INPUT_KEY,
  input="{input}",
  response_key=RESPONSE_KEY,
  response="{response}",
  end_key=END_KEY
)

In [10]:
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['Review of Mobile and Computer Accessories', 'Sentiment', 'Extract Feature Keyword', 'Sentiment_Feature'],
    num_rows: 40
})

In [13]:
def apply_prompt_template(examples):
  instruction = "Find the suitable sentiment and extract features of this given review"
  context = examples.get("Review of Mobile and Computer Accessories")
  response = examples["Sentiment_Feature"]

  if context:
    full_prompt = PROMPT_WITH_INPUT_FORMAT.format(instruction=instruction, response=response, input=context)

  return { "text": full_prompt }

dataset = dataset.map(apply_prompt_template)


Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [14]:
print(dataset["text"][0])

Below is an instruction that describes a task. Write response that appropriately completes the request.

### Instruction:
Find the suitable sentiment and extract features of this given review

Review:
The charging cable frayed within a week.

### Response:
Negative | Frayed, Charging cable

### End


In [15]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model = "meta-llama/Llama-2-7b-hf"

tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model,
    quantization_config=bnb_config,
    device_map="cuda:0",
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['q_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj', 'k_proj', 'v_proj'] # Choose all linear layers from the model
)

In [17]:
from transformers import TrainingArguments

output_dir = "results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 5
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 26
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    ddp_find_unused_parameters=False,
)


In [18]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [19]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [20]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,3.128800
2,2.870600
3,2.494700
4,2.155000
5,1.824300
6,1.447300
7,1.166700
8,0.981400
9,0.716700
10,0.614300


TrainOutput(global_step=26, training_loss=0.8762089002590913, metrics={'train_runtime': 96.8002, 'train_samples_per_second': 4.298, 'train_steps_per_second': 0.269, 'total_flos': 727335820099584.0, 'train_loss': 0.8762089002590913, 'epoch': 10.4})

**Testing**

In [22]:
from transformers import pipeline

In [23]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [27]:
prompt = f"""
Below is an instruction that describes a task. Write response that appropriately completes the request.

### Instruction:
Find the suitable sentiment and extract features of this given review

Review:
The Nokia has two cameras that is not powerful.

### Response:
"""


result = pipe(prompt)
generated_text = result[0]['generated_text']
print(print(generated_text.split("### End")[0]))


Below is an instruction that describes a task. Write response that appropriately completes the request.

### Instruction:
Find the suitable sentiment and extract features of this given review

Review:
The Nokia has two cameras that is not powerful.

### Response:
Negative | Two cameras, Not powerful


None
